In [1]:
import sys

In [2]:
sys.path.append('..')

In [3]:
from jamo import j2hcj, h2j
import pandas as pd
import numpy as np
from konlpy.tag import Twitter

from sklearn.metrics import accuracy_score, f1_score
twitter = Twitter()

/Users/1003874/anaconda/envs/lstm/lib/python3.6/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [4]:
from tensorflow.python.ops.rnn import bidirectional_dynamic_rnn as bi_rnn
from tensorflow.contrib.rnn import BasicLSTMCell
from utils.prepare_data import *
import time
from utils.model_helper import *

/Users/1003874/anaconda/envs/lstm/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
def testcase_shuffle_data_load():
    df = pd.read_csv('/Users/1003874/bsdev/Text-classification-with-CNN-RNN-with-Tensorflow/Ch01_Data_load/data/preprocessed_mart_digi_speech_act.csv')
    df['clean_text'] = df['clean_text'].apply(lambda x:x.lower())
    s = df['class'].value_counts()
    cols = s[s>50].index.tolist()

    sub_list = []
    for col in cols:
        sub_list.append(df[df['class'] == col])
    df = pd.concat(sub_list, axis=0)
    df = df.sample(len(df), random_state=10).reset_index(drop=True)

    added_df = pd.read_csv('/Users/1003874/bsdev/Text-classification-with-CNN-RNN-with-Tensorflow/Ch01_Data_load/data/added_dataset.csv',sep='\t')
    added_df['clean_text'] = added_df['sent'].apply(lambda x:x.lower())

    # class 제거
    added_df = added_df[added_df['class'].apply(lambda x: True if x in cols else False)]
    added_df.info()

    # duplicated sent 제거
    added_df = added_df[added_df['clean_text'].apply(lambda x: False if x in df['clean_text'].tolist() else True)]
    added_df.info()

    # train_df merge

    size = len(df)
    rate = 0.2
    train_df = df.iloc[:int(size * (1 - rate))]
    test_df = df.iloc[int(size * (1 - rate)):]

    all_df = pd.concat([train_df[['clean_text', 'class']],
                        test_df[['clean_text', 'class']],
                        added_df[['clean_text', 'class']]])

    all_df = all_df.sample(len(all_df), random_state=10)
    all_df.reset_index(drop=True, inplace=True)

    label_df = pd.get_dummies(all_df['class'])

    train_df = all_df.iloc[1062:]
    test_df = all_df.iloc[:1062]

    cols = label_df.columns

    print(len(train_df), len(test_df))

    TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL = \
        train_df['clean_text'].values, label_df.loc[train_df.index].values, test_df['clean_text'].values, label_df.loc[
            test_df.index].values

    return TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL, cols

In [6]:
def testcase_add_data_load():
    df = pd.read_csv('/Users/1003874/bsdev/Text-classification-with-CNN-RNN-with-Tensorflow/Ch01_Data_load/data/preprocessed_mart_digi_speech_act.csv')
    df['clean_text'] = df['clean_text'].apply(lambda x:x.lower())
    s = df['class'].value_counts()
    cols = s[s>50].index.tolist()

    sub_list = []
    for col in cols:
        sub_list.append(df[df['class'] == col])
    df = pd.concat(sub_list, axis=0)
    df = df.sample(len(df), random_state=10).reset_index(drop=True)

    added_df = pd.read_csv(
        '/Users/1003874/bsdev/Text-classification-with-CNN-RNN-with-Tensorflow/Ch01_Data_load/data/added_dataset.csv',
        sep='\t')
    added_df['clean_text'] = added_df['sent'].apply(lambda x:x.lower())

    # class 제거
    added_df = added_df[added_df['class'].apply(lambda x: True if x in cols else False)]
    added_df.info()

    # duplicated sent 제거
    added_df = added_df[added_df['clean_text'].apply(lambda x: False if x in df['clean_text'].tolist() else True)]
    added_df.info()

    # train_df merge

    size = len(df)
    rate = 0.2
    train_df = df.iloc[:int(size * (1 - rate))]
    test_df = df.iloc[int(size * (1 - rate)):]
    train_df['setlabel'] = 1
    test_df['setlabel'] = 0
    added_df['setlabel'] = 2

    all_df = pd.concat([train_df[['clean_text', 'class', 'setlabel']],
                        test_df[['clean_text', 'class', 'setlabel']],
                        added_df[['clean_text', 'class','setlabel']]])

    all_df.reset_index(drop=True, inplace=True)

    label_df = pd.get_dummies(all_df['class'])

    train_df = all_df[all_df['setlabel'] != 0]
    test_df = all_df[all_df['setlabel'] == 0]

    cols = label_df.columns

    print(len(train_df), len(test_df))

    TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL = \
        train_df['clean_text'].values, label_df.loc[train_df.index].values, test_df['clean_text'].values, label_df.loc[
            test_df.index].values

    return TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL, cols

In [7]:
def digi_data_load():
    df = pd.read_csv('/Users/1003874/bsdev/Text-classification-with-CNN-RNN-with-Tensorflow/Ch01_Data_load/data/preprocessed_mart_digi_speech_act.csv')
    df['clean_text'] = df['clean_text'].apply(lambda x:x.lower())
    s = df['class'].value_counts()
    cols = s[s>50].index.tolist()

    sub_list = []
    for col in cols:
        sub_list.append(df[df['class'] == col])
    df = pd.concat(sub_list, axis=0)
    df = df.sample(len(df), random_state=10).reset_index(drop=True)

    size = len(df)
    rate = 0.2
    train_df = df.iloc[:int(size*(1-rate))]
    test_df = df.iloc[int(size*(1-rate)):]
    label_df = pd.get_dummies(df['class'])
    cols = label_df.columns
    print(len(train_df), len(test_df))

    TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL = \
        train_df['clean_text'].values, label_df.loc[train_df.index].values, test_df['clean_text'].values, label_df.loc[test_df.index].values

    return TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL, cols

In [8]:
class ABLSTM(object):
    def __init__(self, config):
        self.max_len = config["max_len"]
        self.hidden_size = config["hidden_size"]
        self.vocab_size = config["vocab_size"]
        self.embedding_size = config["embedding_size"]
        self.n_class = config["n_class"]
        self.learning_rate = config["learning_rate"]

        # placeholder
        self.x = tf.placeholder(tf.int32, [None, self.max_len])
        self.label = tf.placeholder(tf.int32, [None])
        self.keep_prob = tf.placeholder(tf.float32)

    def build_graph(self):
        print("building graph")
        # Word embedding
        embeddings_var = tf.Variable(tf.random_uniform([self.vocab_size, self.embedding_size], -1.0, 1.0),
                                     trainable=True)
        batch_embedded = tf.nn.embedding_lookup(embeddings_var, self.x)

        rnn_outputs, _ = bi_rnn(BasicLSTMCell(self.hidden_size),
                                BasicLSTMCell(self.hidden_size),
                                inputs=batch_embedded, dtype=tf.float32)

        fw_outputs, bw_outputs = rnn_outputs

        W = tf.Variable(tf.random_normal([self.hidden_size], stddev=0.1))
        H = fw_outputs + bw_outputs  # (batch_size, seq_len, HIDDEN_SIZE)
        M = tf.tanh(H)  # M = tanh(H)  (batch_size, seq_len, HIDDEN_SIZE)

        self.alpha = tf.nn.softmax(tf.reshape(tf.matmul(tf.reshape(M, [-1, self.hidden_size]),
                                                        tf.reshape(W, [-1, 1])),
                                              (-1, self.max_len)))  # batch_size x seq_len
        r = tf.matmul(tf.transpose(H, [0, 2, 1]),
                      tf.reshape(self.alpha, [-1, self.max_len, 1]))
        r = tf.squeeze(r)
        h_star = tf.tanh(r)  # (batch , HIDDEN_SIZE

        h_drop = tf.nn.dropout(h_star, self.keep_prob)

        # Fully connected layer（dense layer)
        FC_W = tf.Variable(tf.truncated_normal([self.hidden_size, self.n_class], stddev=0.1))
        FC_b = tf.Variable(tf.constant(0., shape=[self.n_class]))
        y_hat = tf.nn.xw_plus_b(h_drop, FC_W, FC_b)

        self.loss = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y_hat, labels=self.label))

        # prediction
        self.prediction = tf.argmax(tf.nn.softmax(y_hat), 1)

        # optimization
        loss_to_minimize = self.loss
        tvars = tf.trainable_variables()
        gradients = tf.gradients(loss_to_minimize, tvars, aggregation_method=tf.AggregationMethod.EXPERIMENTAL_TREE)
        grads, global_norm = tf.clip_by_global_norm(gradients, 1.0)

        self.global_step = tf.Variable(0, name="global_step", trainable=False)
        self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate)
        self.train_op = self.optimizer.apply_gradients(zip(grads, tvars), global_step=self.global_step,
                                                       name='train_step')
        print("graph built successfully!")


In [9]:
TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL, cols = digi_data_load()
#TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL, cols = testcase_add_data_load()
#TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL, cols = testcase_shuffle_data_load()

4244 1062


In [10]:
train_intent = np.apply_along_axis(np.argmax,1,TRAIN_LABEL)
test_intent = np.apply_along_axis(np.argmax,1,TEST_LABEL)

In [11]:
%%time
train_doc = TRAIN_DOC.tolist()
test_doc = TEST_DOC.tolist()

#train_doc = pd.Series(list(map(lambda x: ' '.join(twitter.morphs(x)), train_doc)))
#test_doc = pd.Series(list(map(lambda x: ' '.join(twitter.morphs(x)), test_doc)))
train_doc = pd.Series(list(map(lambda x: ' '.join(list(x.replace(' ','ⓢ'))), train_doc)))
test_doc = pd.Series(list(map(lambda x: ' '.join(list(x.replace(' ','ⓢ'))), test_doc)))

CPU times: user 16.4 ms, sys: 1.65 ms, total: 18 ms
Wall time: 21.7 ms


In [12]:
y_train = pd.Series(train_intent)
y_test = pd.Series(test_intent)

In [13]:
x_train, x_test, tokenizer = \
        data_preprocessing_v2(train_doc, test_doc, max_len=128)
vocab_size = tokenizer.num_words

In [14]:
tokenizer.word_docs

{'0': 1,
 '1': 84,
 '2': 3,
 '4': 2,
 '5': 2,
 'a': 82,
 'b': 1,
 'c': 6,
 'd': 4,
 'e': 6,
 'f': 5,
 'g': 1,
 'h': 1,
 'i': 38,
 'k': 31,
 'l': 4,
 'm': 8,
 'n': 4,
 'o': 20,
 'p': 57,
 'q': 7,
 'r': 4,
 's': 64,
 't': 25,
 'u': 3,
 'v': 27,
 'w': 1,
 'y': 25,
 'ⓢ': 3546,
 '가': 732,
 '각': 6,
 '간': 43,
 '갈': 6,
 '감': 9,
 '갑': 5,
 '갔': 1,
 '강': 1,
 '갖': 2,
 '같': 30,
 '갚': 1,
 '개': 37,
 '객': 29,
 '갸': 1,
 '거': 170,
 '건': 172,
 '걸': 41,
 '검': 19,
 '겁': 5,
 '것': 42,
 '게': 322,
 '겟': 2,
 '겠': 38,
 '겨': 9,
 '격': 78,
 '결': 111,
 '경': 62,
 '계': 35,
 '고': 351,
 '곡': 1,
 '곤': 2,
 '골': 13,
 '곳': 8,
 '공': 25,
 '과': 10,
 '관': 51,
 '광': 6,
 '괜': 18,
 '교': 65,
 '구': 218,
 '국': 3,
 '군': 8,
 '굵': 1,
 '궁': 26,
 '권': 31,
 '귀': 4,
 '귄': 1,
 '규': 2,
 '균': 1,
 '그': 75,
 '근': 18,
 '글': 3,
 '금': 161,
 '급': 42,
 '긍': 1,
 '기': 172,
 '긴': 2,
 '길': 2,
 '까': 103,
 '깐': 2,
 '깡': 1,
 '깨': 1,
 '꺼': 6,
 '껀': 1,
 '께': 53,
 '꼈': 2,
 '꼬': 1,
 '꼭': 1,
 '꽤': 1,
 '꾸': 23,
 '꿀': 5,
 '꿈': 1,
 '꿔': 9,
 '꿨': 1,
 '뀌': 7,
 '끄': 2

In [ ]:
x_dev = x_test
y_dev = y_test
dev_size = len(x_dev)
test_size = len(x_test)
print("Validation Size: ", dev_size)

config = {
    "max_len": 64,
    "hidden_size": 64,
    "vocab_size": vocab_size,
    "embedding_size": 128,
    "n_class": 33,
    "learning_rate": 1e-3,
    "batch_size": 4,
    "train_epoch": 20
}

classifier = ABLSTM(config)
classifier.build_graph()

Validation Size:  1062
building graph
graph built successfully!


In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
dev_batch = (x_dev, y_dev)
start = time.time()
for e in range(config["train_epoch"]):

    t0 = time.time()
    print("Epoch %d start !" % (e + 1))
    for x_batch, y_batch in fill_feed_dict(x_train, y_train, config["batch_size"]):
        return_dict = run_train_step(classifier, sess, (x_batch, y_batch))
        attn = get_attn_weight(classifier, sess, (x_batch, y_batch))
        # plot the attention weight
        # print(np.reshape(attn, (config["batch_size"], config["max_len"])))
    t1 = time.time()

    print("Train Epoch time:  %.3f s" % (t1 - t0))
    dev_acc = run_eval_step(classifier, sess, dev_batch)
    print("validation accuracy: %.3f " % dev_acc)

print("Training finished, time consumed : ", time.time() - start, " s")
print("Start evaluating:  \n")
cnt = 0
test_acc = 0
for x_batch, y_batch in fill_feed_dict(x_test, y_test, config["batch_size"]):
    acc = run_eval_step(classifier, sess, (x_batch, y_batch))
    test_acc += acc
    cnt += 1

print("Test accuracy : %f %%" % (test_acc / cnt * 100))


Epoch 1 start !
Train Epoch time:  625.326 s
validation accuracy: 0.333 
Epoch 2 start !
Train Epoch time:  621.160 s
validation accuracy: 0.442 
Epoch 3 start !
Train Epoch time:  624.311 s
validation accuracy: 0.492 
Epoch 4 start !
Train Epoch time:  642.462 s
validation accuracy: 0.512 
Epoch 5 start !
Train Epoch time:  632.748 s
validation accuracy: 0.533 
Epoch 6 start !
Train Epoch time:  642.512 s
validation accuracy: 0.558 
Epoch 7 start !
Train Epoch time:  645.349 s
validation accuracy: 0.582 
Epoch 8 start !
Train Epoch time:  601.014 s
validation accuracy: 0.593 
Epoch 9 start !
Train Epoch time:  596.741 s
validation accuracy: 0.597 
Epoch 10 start !
Train Epoch time:  562.877 s
validation accuracy: 0.605 
Epoch 11 start !
Train Epoch time:  584.656 s
validation accuracy: 0.607 
Epoch 12 start !
Train Epoch time:  606.369 s
validation accuracy: 0.612 
Epoch 13 start !


In [ ]:
preds = []
reals = []
attns = []
for x_batch, y_batch in fill_feed_dict(x_test, y_test,1062, isshuffle=False):
    pred = get_prediction(classifier, sess, (x_batch, y_batch))
    attn = get_attn_weight(classifier, sess, (x_batch, y_batch))
    preds += pred.tolist()
    attns += attn.tolist()
    reals += y_batch.tolist()

In [ ]:
test_df = pd.DataFrame({'sent':test_doc,'attn':attns, 'pred':preds, 'real':reals})
test_df

In [ ]:
test_df['real'] = test_df['real'].apply(lambda x: cols[x])
test_df['pred'] = test_df['pred'].apply(lambda x: cols[x])

In [ ]:
f1_score(test_df['real'],test_df['pred'],average='weighted')

In [ ]:
test_df[test_df['real'] == test_df['pred']]

In [5]:
' '.join(list('ㄱㅏㄴㅡ ㄷㅏㄹㅏㅇ'.replace(' ', 'ⓢ')))

'ㄱ ㅏ ㄴ ㅡ ⓢ ㄷ ㅏ ㄹ ㅏ ㅇ'

In [ ]:
data_preprocessing_v2